In [1]:
from transformers import pipeline
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

test_text = "Machine learning is a field of artificial intelligence that uses statistical techniques to give computer systems the ability to 'learn' from data."

print(summarizer(test_text)[0]['summary_text'])


Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

C:\Users\AHINA\anaconda3\envs\papersum\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\AHINA\.cache\huggingface\hub\models--facebook--bart-large-cnn. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu
Your max_length is set to 142, but your input_length is only 27. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)


Machine learning is a field of artificial intelligence that uses statistical techniques to give computer systems the ability to 'learn' from data. Machine learning is the study of how computers learn by analysing data. For more information on machine learning, visit machinelearning.org.uk. To learn more about machine learning on CNN.com, click here.


In [3]:
!pip install PyMuPDF transformers sentencepiece ipywidgets


  Using cached pymupdf-1.26.0-cp39-abi3-win_amd64.whl.metadata (3.4 kB)
Using cached pymupdf-1.26.0-cp39-abi3-win_amd64.whl (18.5 MB)


In [21]:
from IPython.display import display
import ipywidgets as widgets

upload = widgets.FileUpload(accept='.pdf', multiple=False)
display(upload)


FileUpload(value=(), accept='.pdf', description='Upload')

In [32]:
import fitz  # PyMuPDF
import ipywidgets as widgets
from IPython.display import display, clear_output

upload = widgets.FileUpload(accept='.pdf', multiple=False)
display(upload)

output = widgets.Output()
display(output)

def extract_pdf_text(file_content):
    try:
        with open("temp_uploaded.pdf", "wb") as f:
            f.write(file_content)
        doc = fitz.open("temp_uploaded.pdf")
        full_text = ""
        for page in doc:
            full_text += page.get_text()
        return full_text
    except Exception as e:
        return f"Error extracting PDF text: {e}"

def on_upload_change(change):
    output.clear_output()
    try:
        if upload.value:
            print("File uploaded, extracting text...")

            # Handle tuple or dict
            if isinstance(upload.value, tuple):
                uploaded_file = upload.value[0]
            elif isinstance(upload.value, dict):
                uploaded_file = list(upload.value.values())[0]
            else:
                raise ValueError(f"Unexpected upload.value type: {type(upload.value)}")

            content = uploaded_file['content']
            text = extract_pdf_text(content)
            with output:
                print("✅ PDF Loaded. Extracted Text Preview:\n")
                print(text[:2000])
        else:
            with output:
                print("No file uploaded yet.")
    except Exception as e:
        with output:
            print(f"Error in upload handler: {e}")


upload.observe(on_upload_change, names='value')


FileUpload(value=(), accept='.pdf', description='Upload')

Output()

In [33]:
import fitz  # PyMuPDF
from transformers import pipeline
import ipywidgets as widgets
from IPython.display import display, clear_output


In [34]:
def extract_pdf_text(file_content):
    with open("temp_uploaded.pdf", "wb") as f:
        f.write(file_content)
    doc = fitz.open("temp_uploaded.pdf")
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    return full_text


In [35]:
print("Loading summarization model (this may take a minute)...")
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)  # device=-1 => CPU
print("Model loaded!")


Loading summarization model (this may take a minute)...


Device set to use cpu


Model loaded!


In [36]:
upload = widgets.FileUpload(accept=".pdf", multiple=False)
output = widgets.Output()

display(upload, output)


FileUpload(value=(), accept='.pdf', description='Upload')

Output()

In [37]:
def summarize_text(text, max_chunk=1000):
    # Split text into chunks of max_chunk size (approximate)
    chunks = [text[i:i+max_chunk] for i in range(0, len(text), max_chunk)]
    summaries = []
    for chunk in chunks:
        summary = summarizer(chunk)[0]['summary_text']
        summaries.append(summary)
    return " ".join(summaries)

def on_upload_change(change):
    with output:
        clear_output()
        try:
            if upload.value:
                # Support both tuple and dict types for upload.value
                if isinstance(upload.value, tuple):
                    uploaded_file = upload.value[0]
                elif isinstance(upload.value, dict):
                    uploaded_file = list(upload.value.values())[0]
                else:
                    raise ValueError(f"Unexpected upload.value type: {type(upload.value)}")

                file_content = uploaded_file['content']
                print("Extracting text from PDF...")
                text = extract_pdf_text(file_content)
                print(f"Extracted text length: {len(text)} characters.\n")
                print("Summarizing text...")
                summary = summarize_text(text)
                print("\nSummary:")
                print(summary)
            else:
                print("No file uploaded yet.")
        except Exception as e:
            print(f"Error: {e}")

upload.observe(on_upload_change, names='value')


In [38]:
import fitz
from transformers import pipeline
import ipywidgets as widgets
from IPython.display import display, clear_output

# Summarization pipeline (CPU)
summarizer = pipeline("summarization", model="facebook/bart-large-cnn", device=-1)

upload = widgets.FileUpload(accept=".pdf", multiple=False)
output = widgets.Output()

def extract_pdf_text(file_content):
    with open("temp_uploaded.pdf", "wb") as f:
        f.write(file_content)
    doc = fitz.open("temp_uploaded.pdf")
    full_text = ""
    for page in doc:
        full_text += page.get_text()
    return full_text

def summarize_text(text, max_chunk=1000):
    chunks = [text[i:i+max_chunk] for i in range(0, len(text), max_chunk)]
    summaries = []
    for chunk in chunks:
        summaries.append(summarizer(chunk)[0]['summary_text'])
    return " ".join(summaries)

def on_upload_change(change):
    with output:
        clear_output()
        if not upload.value:
            print("No file uploaded.")
            return
        try:
            # Support dict or tuple upload.value
            if isinstance(upload.value, dict):
                uploaded_file = list(upload.value.values())[0]
            else:
                uploaded_file = upload.value[0]

            content = uploaded_file['content']
            print("Extracting text...")
            text = extract_pdf_text(content)
            print(f"Extracted {len(text)} characters.")
            print("Summarizing...")
            summary = summarize_text(text)
            print("\nSummary:\n", summary)
        except Exception as e:
            print(f"Error: {e}")

upload.observe(on_upload_change, names='value')

display(upload, output)


Device set to use cpu


FileUpload(value=(), accept='.pdf', description='Upload')

Output()

In [39]:

print(f"\nSummary length: {len(summary)} characters")



Summary length: 309 characters
